In [50]:
import numpy as np

sigma = 0.32
r = 0.04
T = 1
K = 100
S0 = 100

def euro_option_price (N, optiontype):
    ST = np.zeros(N+1)
    V = np.zeros(N+1)
    
    deltat = T/N
    u = np.exp(sigma * np.sqrt(deltat))
    d = 1/u
    p = (np.exp(r * deltat)-d)/(u-d)
    
    j = np.arange(0, N+1)
    ST = S0 * (u**j) * (d**(N-j))
    
    # When optiontype is 0, it represents call option. Otherwise, it is put.
    if optiontype == 0:
        V = np.maximum(ST - K, 0)
    else:
        V = np.maximum(K - ST, 0)
    
    for n in np.arange(N, 0, -1):
        V = np.exp(-r * deltat) * (p * V[1:n+1] + (1-p) * V[0:n])
    
    value = V[0]
    return value

    
    
def amer_option_price (N, optiontype):
    ST = np.zeros(N+1)
    V = np.zeros(N+1)
    
    deltat = T/N
    u = np.exp(sigma * np.sqrt(deltat))
    d = 1/u
    p = (np.exp(r * deltat)-d)/(u-d)
    
    j = np.arange(0, N+1)
    ST = S0 * (u**j) * (d**(N-j))
    
    # When optiontype is 0, it represents call option. Otherwise, it is put.
    if optiontype == 0:
        V = np.maximum(ST - K, 0)
    else:
        V = np.maximum(K - ST, 0)
    
    for n in np.arange(N, 0, -1):
        ST = ST[0:n] * u
        V = np.exp(-r * deltat) * (p * V[1:n+1] + (1-p) * V[0:n])
        if optiontype == 0:
            Stop_payoff = np.maximum(ST - K, 0)
        else:
            Stop_payoff = np.maximum(K - ST, 0)
        V = np.maximum(Stop_payoff, V)
        
    value = V[0]
    return value


In [26]:
#adapted from Octave's financial toolkit
from scipy.stats import norm
import numpy as np
    
def blsprice(Price, Strike, Rate, Time, Volatility):
    sigma_sqrtT = Volatility * np.sqrt (Time)

    d1 = 1 / sigma_sqrtT * (np.log(Price / Strike) + (Rate + Volatility**2 / 2) * Time)
    d2 = d1 - sigma_sqrtT

    phi1 = norm.cdf(d1)
    phi2 = norm.cdf(d2)
    disc = np.exp (-Rate * Time)
    F    = Price * np.exp ((Rate) * Time)

    Call = disc * (F * phi1 - Strike * phi2)
    Put  = disc * (Strike * (1 - phi2) + F * (phi1 - 1))
    return Call, Put


In [45]:
# For European Call Table
theo_call = blsprice(S0, K, r, T, sigma)[0]

print('delat_t\t\tValue\t\t\tChange\t\t\tRatio')
Value_eurocall = np.zeros(5)
Value_eurocall[0] = euro_option_price(500, 0)
Value_eurocall[1] = euro_option_price(1000, 0)
Value_eurocall[2] = euro_option_price(2000, 0)
Value_eurocall[3] = euro_option_price(4000, 0)
Value_eurocall[4] = euro_option_price(8000, 0)

deltat_table = np.zeros(5)
deltat_table[0] = T/500
deltat_table[1] = T/1000
deltat_table[2] = T/2000
deltat_table[3] = T/4000
deltat_table[4] = T/8000

valuechange_eurocall = np.zeros(5)
valuechange_eurocall[1] = Value_eurocall[1]-Value_eurocall[0]
valuechange_eurocall[2] = Value_eurocall[2]-Value_eurocall[1]
valuechange_eurocall[3] = Value_eurocall[3]-Value_eurocall[2]
valuechange_eurocall[4] = Value_eurocall[4]-Value_eurocall[3]

ratio_eurocall = np.zeros(5)
ratio_eurocall[2] = valuechange_eurocall[1]/valuechange_eurocall[2]
ratio_eurocall[3] = valuechange_eurocall[2]/valuechange_eurocall[3]
ratio_eurocall[4] = valuechange_eurocall[3]/valuechange_eurocall[4]


for i in range(5):
    if len(str(deltat_table[i])) < 7:
        if valuechange_eurocall[i] == 0:
            print(str(deltat_table[i]) + " \t\t" + str(Value_eurocall[i]) + "\t" + ' ' + "\t\t\t" + ' ')
        elif ratio_eurocall[i] == 0:
            print(str(deltat_table[i]) + " \t\t" + str(Value_eurocall[i]) + "\t" + str(valuechange_eurocall[i]) + "\t\t\t" + ' ')
        else: 
            print(str(deltat_table[i]) + " \t\t" + str(Value_eurocall[i]) + "\t" + str(valuechange_eurocall[i]) + "\t" + str(ratio_eurocall[i]))  
    else:
        print(str(deltat_table[i]) + " \t" + str(Value_eurocall[i]) + "\t" + str(valuechange_eurocall[i]) + "\t" + str(ratio_eurocall[i]))
print('exact\t\t'+str(theo_call))        

delat_t		Value			Change			Ratio
0.002 		14.513319919417471	 			 
0.001 		14.516457291797295	0.0031373723798235886			 
0.0005 		14.518026273384256	0.0015689815869617263	1.999623453771049
0.00025 	14.518810837859961	0.000784564475704741	1.9998121703794665
0.000125 	14.51920313853851	0.00039230067854845174	1.9999059869274278
exact		14.519595451443333


In [49]:
# For European Put Table
theo_put = blsprice(S0, K, r, T, sigma)[1]

print('delat_t\t\tValue\t\t\tChange\t\t\tRatio')
Value_europut = np.zeros(5)
Value_europut[0] = euro_option_price(500, 1)
Value_europut[1] = euro_option_price(1000, 1)
Value_europut[2] = euro_option_price(2000, 1)
Value_europut[3] = euro_option_price(4000, 1)
Value_europut[4] = euro_option_price(8000, 1)

deltat_table = np.zeros(5)
deltat_table[0] = T/500
deltat_table[1] = T/1000
deltat_table[2] = T/2000
deltat_table[3] = T/4000
deltat_table[4] = T/8000

valuechange_europut = np.zeros(5)
valuechange_europut[1] = Value_europut[1]-Value_europut[0]
valuechange_europut[2] = Value_europut[2]-Value_europut[1]
valuechange_europut[3] = Value_europut[3]-Value_europut[2]
valuechange_europut[4] = Value_europut[4]-Value_europut[3]

ratio_europut = np.zeros(5)
ratio_europut[2] = valuechange_europut[1]/valuechange_europut[2]
ratio_europut[3] = valuechange_europut[2]/valuechange_europut[3]
ratio_europut[4] = valuechange_europut[3]/valuechange_europut[4]


for i in range(5):
    if len(str(deltat_table[i])) < 7:
        if valuechange_europut[i] == 0:
            print(str(deltat_table[i]) + " \t\t" + str(Value_europut[i]) + "\t" + ' ' + "\t\t\t" + ' ')
        elif ratio_europut[i] == 0:
            print(str(deltat_table[i]) + " \t\t" + str(Value_europut[i]) + "\t" + str(valuechange_europut[i]) + "\t\t\t" + ' ')
        else: 
            print(str(deltat_table[i]) + " \t\t" + str(Value_europut[i]) + "\t" + str(valuechange_europut[i]) + "\t" + str(ratio_europut[i]))  
    else:
        print(str(deltat_table[i]) + " \t" + str(Value_europut[i]) + "\t" + str(valuechange_europut[i]) + "\t" + str(ratio_europut[i]))
print('exact\t\t'+str(theo_put))          

delat_t		Value			Change			Ratio
0.002 		10.592263834646982	 			 
0.001 		10.595401207019853	0.003137372372870928			 
0.0005 		10.596970188631333	0.0015689816114807797	1.9996234180908763
0.00025 	10.597754753131191	0.0007845644998578649	1.99981214006627
0.000125 	10.5981470537607	0.0003923006295085685	1.9999062984953195
exact		10.598539366675652


In [51]:
# For American Put Table

print('delat_t\t\tValue\t\t\tChange\t\t\tRatio')
Value_amerput = np.zeros(5)
Value_amerput[0] = amer_option_price(500, 1)
Value_amerput[1] = amer_option_price(1000, 1)
Value_amerput[2] = amer_option_price(2000, 1)
Value_amerput[3] = amer_option_price(4000, 1)
Value_amerput[4] = amer_option_price(8000, 1)

deltat_table = np.zeros(5)
deltat_table[0] = T/500
deltat_table[1] = T/1000
deltat_table[2] = T/2000
deltat_table[3] = T/4000
deltat_table[4] = T/8000

valuechange_amerput = np.zeros(5)
valuechange_amerput[1] = Value_amerput[1]-Value_amerput[0]
valuechange_amerput[2] = Value_amerput[2]-Value_amerput[1]
valuechange_amerput[3] = Value_amerput[3]-Value_amerput[2]
valuechange_amerput[4] = Value_amerput[4]-Value_amerput[3]

ratio_amerput = np.zeros(5)
ratio_amerput[2] = valuechange_amerput[1]/valuechange_amerput[2]
ratio_amerput[3] = valuechange_amerput[2]/valuechange_amerput[3]
ratio_amerput[4] = valuechange_amerput[3]/valuechange_amerput[4]


for i in range(5):
    if len(str(deltat_table[i])) < 7:
        if valuechange_amerput[i] == 0:
            print(str(deltat_table[i]) + " \t\t" + str(Value_amerput[i]) + "\t" + ' ' + "\t\t\t" + ' ')
        elif ratio_europut[i] == 0:
            print(str(deltat_table[i]) + " \t\t" + str(Value_amerput[i]) + "\t" + str(valuechange_amerput[i]) + "\t\t\t" + ' ')
        else: 
            print(str(deltat_table[i]) + " \t\t" + str(Value_amerput[i]) + "\t" + str(valuechange_amerput[i]) + "\t" + str(ratio_amerput[i]))  
    else:
        print(str(deltat_table[i]) + " \t" + str(Value_amerput[i]) + "\t" + str(valuechange_amerput[i]) + "\t" + str(ratio_amerput[i]))


delat_t		Value			Change			Ratio
0.002 		10.991823139809181	 			 
0.001 		10.99345810026476	0.001634960455579204			 
0.0005 		10.994280111871603	0.0008220116068429206	1.9889749025060068
0.00025 	10.994684282443759	0.0004041705721551381	2.0338234979844034
0.000125 	10.994886666120573	0.00020238367681457703	1.9970512371184819
